# Information Retrieval 
## Sonali Agarkar


# Task 1- **Vertical Search Engine**
###Import Basic Libraries:

In [ ]:
!pip install schedule


In [ ]:
#Importing libraries
import pandas as pd
import numpy as np
import schedule
from bs4 import BeautifulSoup
import requests
import re
import os
from csv import DictWriter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display
import warnings
warnings.simplefilter("ignore")
import schedule
import time
import seaborn as sns;sns.set(style="white")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Libraries for text preprocessing
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')

import string
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### CRAWLER

In [ ]:
def mycrawler():
  count=0 #Counting number of publications
  col_name = ['Publication Link','Title','Authors', 'Date', 'Journal Title'] #information to retrieve
  url_to_crawl='https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences'
  extended_url='https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences/publications'
  head='https://pureportal.coventry.ac.uk'
  Next = True #to check if next page exist or not
  res_out=["ContributionToJournal","ContributionToConference"
                                        ,"ContributionToBookAnthology","BookAnthology",\
                                        "ContributionToPeriodical","OtherContribution",\
                                        "Thesis","NonTextual","Patent"]
  print(extended_url)
  info = {'Publication Link':'Publication link',
              'Title':"Title",
              'Authors':'Author(s)',
              'Date': 'Publication Date',
              'Journal Title': 'Journal Title'}
  with open('info.csv', 'a+') as outfile:
    dict_writer = DictWriter(outfile, fieldnames=col_name)
    dict_writer.writerow(info)
    outfile.close()
   
  while Next:
    time.sleep(1)
    req = requests.get(extended_url)
    bs = BeautifulSoup(req.content, 'html.parser')
    Next = bs.find("li",class_="next")
    if Next == None:
      break;
    
    link_ext = Next.find("a")["href"]
    print(link_ext)

    if count==0:
      extended_url=extended_url
    else:
      extended_url=head+link_ext

    print(extended_url)
    req = requests.get(extended_url)
    bs= BeautifulSoup(req.content, 'html.parser')
    contents = bs.find_all("div",{'class':"result-container"})
    for content in contents:
      publink=content.find("a",{'rel':res_out})
      pub_title = publink.text
      pub_link = publink['href']
      print("Publication Title *****",pub_title)
      print("Publication Link *****",pub_link)
      pub_author = content.find_all("a",{'rel':"Person"})
      # print("Authors:******",pub_author)
      
      all_authors={}
      count=count+1
      for author in pub_author:
        all_authors[author.text] = author['href']
      print("Authors:******",all_authors)

      date = content.find("span",{'class':["date"]}).text
      print("Publication Date:******",date)

      # citation_journal_title
      try:
         journal_title = content.find("a",{'rel':["Journal"]}).text()
      except:
        journal_title = "Journal Title Not Present"
      print("Citation Journal Title:******",journal_title)

        
      info = {'Publication Link':pub_link,
              'Title':pub_title,
              'Authors':all_authors,
              'Date': date,
              'Journal Title': journal_title}
      if  len(all_authors) > 0:
    
        with open('info.csv', 'a+') as outfile:
              dict_writer = DictWriter(outfile, fieldnames=col_name)
              dict_writer.writerow(info)
              outfile.close()
      print(count)

if os.path.exists("info.csv"):
  os.remove("info.csv")

In [ ]:
#schedule.every(1).friday.at("18:32").do(mycrawler())
mycrawler()

https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences/publications
/en/organisations/school-of-life-sciences/publications/?page=1
https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences/publications
Publication Title ***** A Comparison of Physical Activity Between Home-Based and Centre-Based Pulmonary Rehabilitation: A Randomised Controlled Secondary Analysis
Publication Link ***** https://pureportal.coventry.ac.uk/en/publications/a-comparison-of-physical-activity-between-home-based-and-centre-b
Authors:****** {'Horton, E.': 'https://pureportal.coventry.ac.uk/en/persons/elizabeth-horton', 'Sewell, L.': 'https://pureportal.coventry.ac.uk/en/persons/louise-sewall'}
Publication Date:****** 26 Oct 2021
Citation Journal Title:****** Journal Title Not Present
1
Publication Title ***** Actual and perceived motor competence mediate the relationship between physical fitness and technical skill performance in young soccer players
Publication Link ***** htt

In [ ]:
df= pd.read_csv("info.csv")
print("Orignal dataframe have {} row.\n" .format(df.shape[0]))
df = df.drop_duplicates(subset=['Title'], keep='first', ignore_index=True)
print("After dropping duplicated dataframe have {} row.\n" .format(df.shape[0]))
df

Orignal dataframe have 515 row.

After dropping duplicated dataframe have 515 row.



,Publication link,Title,Author(s),Publication Date,Journal Title
0,https://pureportal.coventry.ac.uk/en/publicati...,A Comparison of Physical Activity Between Home...,"{'Horton, E.': 'https://pureportal.coventry.ac...",26 Oct 2021,Journal Title Not Present
1,https://pureportal.coventry.ac.uk/en/publicati...,Actual and perceived motor competence mediate ...,"{'Duncan, M.': 'https://pureportal.coventry.ac...",11 Jul 2021,Journal Title Not Present
2,https://pureportal.coventry.ac.uk/en/publicati...,Acute Hormonal Response to Kettlebell Swing Ex...,"{'Renshaw, D.': 'https://pureportal.coventry.a...",1 Apr 2021,Journal Title Not Present
3,https://pureportal.coventry.ac.uk/en/publicati...,Associations Between Self-Reported Sleep Durat...,"{'Roden, L. C.': 'https://pureportal.coventry....",Jul 2021,Journal Title Not Present
4,https://pureportal.coventry.ac.uk/en/publicati...,Blood cancer care in a resource limited settin...,"{'Lin, T. T.': 'https://pureportal.coventry.ac...",17 Sep 2021,Journal Title Not Present
...,...,...,...,...,...
510,https://pureportal.coventry.ac.uk/en/publicati...,Reconstruction of the Bone-Bone Marrow Organ b...,"{'Yeates, L.': 'https://pureportal.coventry.ac...",Jan 1993,Journal Title Not Present
511,https://pureportal.coventry.ac.uk/en/publicati...,Method of preparative genes expression in the ...,"{'Morozov, I.': 'https://pureportal.coventry.a...",15 Jan 1992,Journal Title Not Present
512,https://pureportal.coventry.ac.uk/en/publicati...,The pathogenicity of two porcine rotaviruses d...,"{'Burke, B.': 'https://pureportal.coventry.ac....",Nov 1992,Journal Title Not Present
513,https://pureportal.coventry.ac.uk/en/publicati...,Method for preparative expression of genes in ...,"{'Morozov, I.': 'https://pureportal.coventry.a...",12 Dec 1990,Journal Title Not Present


In [ ]:
df['Title'].head(10)

0    A Comparison of Physical Activity Between Home...
1    Actual and perceived motor competence mediate ...
2    Acute Hormonal Response to Kettlebell Swing Ex...
3    Associations Between Self-Reported Sleep Durat...
4    Blood cancer care in a resource limited settin...
5    British Soccer Academy Personnel Perceive Psyc...
6    Changes in joint kinematics and dynamic postur...
7    Comparative analysis of reported physical acti...
8    Cross-cultural comparison of fundamental movem...
9    Delivery Approaches Within Exercise Referral S...
Name: Title, dtype: object

In [ ]:
def preprocess(title):
  title = title.lower()
  title = re.sub(r'[%s]' % re.escape(string.punctuation), '', title)
  title = re.sub(r'[0-9]', '', title)
  title = re.sub(r'@\w+','',title)
  title = re.sub(r'\s{2,}', '', title)
  print(title)
  sequence = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  text = sequence.sub('', title)
  return (text)

In [ ]:
corpus= df['Title']

In [ ]:
cleantitles=[] 
for title in corpus:
  cleantitles.append(preprocess(title))


a comparison of physical activity between homebased and centrebased pulmonary rehabilitation a randomised controlled secondary analysis
actual and perceived motor competence mediate the relationship between physical fitness and technical skill performance in young soccer players
acute hormonal response to kettlebell swing exercise differs depending on load even when total work is normalized
associations between selfreported sleep duration and mortality in employed individuals systematic review and metaanalysis
blood cancer care in a resource limited setting during the covid outbreak a single center experience from sri lanka
british soccer academy personnel perceive psychological and technicaltactical attributes as the most important contributors to development
changes in joint kinematics and dynamic postural stability with free and restricted arm movements in children
comparative analysis of reported physical activity from leisure centres’ members versus the general population in spain

In [ ]:
def stemming(title):
  list1=[]
  num=0
  for word in title:
    sentence=[]
    tokens=word_tokenize(word)
    stemwords=[]
    for token in tokens:
      stemwords.append(stemmer.stem(token))
    sentence=' '.join(stemwords)
    list1.insert(num,sentence)
    num=num+1
      
  return (list1)

In [ ]:
cleantitles=stemming(cleantitles)
cleantitles

['comparison physic activ homebas centrebas pulmonari rehabilit randomis control secondari analysi',
 'actual perceiv motor compet mediat relationship physic fit technic skill perform young soccer player',
 'acut hormon respons kettlebel swing exercis differ depend load even total work normal',
 'associ selfreport sleep durat mortal employ individu systemat review metaanalysi',
 'blood cancer care resourc limit set covid outbreak singl center experi sri lanka',
 'british soccer academi personnel perceiv psycholog technicaltact attribut import contributor develop',
 'chang joint kinemat dynam postur stabil free restrict arm movement children',
 'compar analysi report physic activ leisur centr ’ member versus general popul spain',
 'crosscultur comparison fundament movement skill intoyear old children england china',
 'deliveri approach within exercis referr scheme survey current practic england',
 'develop power athlet partpract applic',
 'resist sled tow improv physic qualiti elit yout

##TFIDF

In [ ]:
tfvec = TfidfVectorizer()   
vectors = tfvec.fit_transform(cleantitles) #this would be used to find cosine similarity for ranking
# print(vectors.shape)
col = tfvec.get_feature_names()
# print(col)
denvec = vectors.todense()
veclist = denvec.tolist()
df2 = pd.DataFrame(veclist, columns=col)

In [ ]:
print(df2.shape)
df2.head()

(515, 1924)


,abil,absolut,abund,academ,academi,acceler,acceleromet,accelerometeri,accelerometri,access,acclim,accord,account,accumul,acetohydroxyacid,achiev,achill,acid,acidbas,acl,acn,across,acrossmonth,act,actigraph,action,activ,activitypreliminari,actiwatchin,actual,aculeatus,acut,acyl,adam,adamt,adapt,addit,address,adenosin,adenovirus,...,walk,water,wave,wavesrippl,way,wearabl,wearsit,week,weekday,weekend,weight,wellb,wellington,west,wet,wheel,wheelchair,whilst,white,whole,winter,within,without,wnk,women,work,workload,worldwid,wrap,wrist,write,year,yeast,young,youth,yr,zimbabw,zr,zumba,ênfase
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.186758,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.334353,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.276632,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.215562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.308059,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


##Query Processing

In [ ]:
query = input("Enter Your Query: ")

Enter Your Query: thermoregulation


In [ ]:
tfidf_query = [query]
tfidf_query = preprocess(tfidf_query[0])
tfidf_query=stemming([tfidf_query])
print("Query Tokens",tfidf_query)

thermoregulation
Query Tokens ['thermoregul']


In [ ]:
query_tfidf_vector = tfvec.transform(tfidf_query)
query_tfidf_vector.toarray().shape

(1, 1924)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(
    query_tfidf_vector,         #TF-IDF vector of query    
    vectors)                   #vectors is TF-IDF vectors for indexed documents
cos_sim

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [ ]:
top_n_similar_title = 10

simalarity_indices=( sorted( [(x,i) for (i,x) in enumerate(cos_sim[0])], reverse=True )[:top_n_similar_title] )

similarity_list = [index[0] for index in simalarity_indices]
index_list = [index[1] for index in simalarity_indices]
# print(index_list)
# print(similarity_list)
data = df.iloc[index_list]
data["cosine_similarity"] = similarity_list
dfStyler = data.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
display(data)

,Publication link,Title,Author(s),Publication Date,Journal Title,cosine_similarity
155,https://pureportal.coventry.ac.uk/en/publicati...,Thermoregulation during upper body exercise,"{'Price, M. J.': 'https://pureportal.coventry....",1 Jan 2019,Journal Title Not Present,0.594749
212,https://pureportal.coventry.ac.uk/en/publicati...,Quadriplegia: The challenge of thermoregulation,"{'Price, M. J.': 'https://pureportal.coventry....",Jul 2018,Journal Title Not Present,0.551494
228,https://pureportal.coventry.ac.uk/en/publicati...,Thermoregulation following spinal cord injury,"{'Price, M. J.': 'https://pureportal.coventry....",2018,Journal Title Not Present,0.487212
178,https://pureportal.coventry.ac.uk/en/publicati...,Classification of spinal cord injury: The role...,"{'Price, M. J.': 'https://pureportal.coventry....",Jul 2018,Journal Title Not Present,0.428381
514,https://pureportal.coventry.ac.uk/en/publicati...,Method for preparative expression of genes in ...,"{'Morozov, I.': 'https://pureportal.coventry.a...",28 Jun 1990,Journal Title Not Present,0.000000
513,https://pureportal.coventry.ac.uk/en/publicati...,Method for preparative expression of genes in ...,"{'Morozov, I.': 'https://pureportal.coventry.a...",12 Dec 1990,Journal Title Not Present,0.000000
512,https://pureportal.coventry.ac.uk/en/publicati...,The pathogenicity of two porcine rotaviruses d...,"{'Burke, B.': 'https://pureportal.coventry.ac....",Nov 1992,Journal Title Not Present,0.000000
511,https://pureportal.coventry.ac.uk/en/publicati...,Method of preparative genes expression in the ...,"{'Morozov, I.': 'https://pureportal.coventry.a...",15 Jan 1992,Journal Title Not Present,0.000000
510,https://pureportal.coventry.ac.uk/en/publicati...,Reconstruction of the Bone-Bone Marrow Organ b...,"{'Yeates, L.': 'https://pureportal.coventry.ac...",Jan 1993,Journal Title Not Present,0.000000
509,https://pureportal.coventry.ac.uk/en/publicati...,Nucleotide sequences of the VP4 and VP7 genes ...,"{'Burke, B.': 'https://pureportal.coventry.ac....",Nov 1993,Journal Title Not Present,0.000000


In [ ]:
print("Search Query:", query)
print()
for index, row in data.iterrows():
  print("Publication Link:  ", row["Publication link"])
  print("Publication Title: ", row["Title"])
  print("Publication Author:", row["Author(s)"])
  print("Publication Date:", row["Publication Date"])
  print("Journal Title:", row["Journal Title"])
  print("Cosine Similarty:", row["cosine_similarity"])
  print()

Search Query: thermoregulation

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/thermoregulation-during-upper-body-exercise
Publication Title:  Thermoregulation during upper body exercise
Publication Author: {'Price, M. J.': 'https://pureportal.coventry.ac.uk/en/persons/mike-price'}
Publication Date: 1 Jan 2019
Journal Title: Journal Title Not Present
Cosine Similarty: 0.5947487828470245

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/quadriplegia-the-challenge-of-thermoregulation
Publication Title:  Quadriplegia: The challenge of thermoregulation
Publication Author: {'Price, M. J.': 'https://pureportal.coventry.ac.uk/en/persons/mike-price'}
Publication Date: Jul 2018
Journal Title: Journal Title Not Present
Cosine Similarty: 0.5514940333004401

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/thermoregulation-following-spinal-cord-injury
Publication Title:  Thermoregulation following spinal cord injury
Publication Autho

#Combining entire code for query processor
##without displaying intermediate results

In [ ]:
query = input("Enter Your Query: ")
tfidf_query = [query]
tfidf_query = preprocess(tfidf_query[0])
tfidf_query=stemming([tfidf_query])
# print("Query Tokens",tfidf_query)
query_tfidf_vector = tfvec.transform(tfidf_query)
query_tfidf_vector.toarray().shape
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(
    query_tfidf_vector,
    vectors)
top_n_similar_title = 5

simalarity_indices=( sorted( [(x,i) for (i,x) in enumerate(cos_sim[0])], reverse=True )[:top_n_similar_title] )

similarity_list = [index[0] for index in simalarity_indices]
index_list = [index[1] for index in simalarity_indices]
# print(index_list)
# print(similarity_list)
data = df.iloc[index_list]
data["cosine_similarity"] = similarity_list
dfStyler = data.style.set_properties(**{'text-align': 'left'})
dfStyler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
# display(data)
print("Search Query:", query)
print()
for index, row in data.iterrows():
  print("Publication Link:  ", row["Publication link"])
  print("Publication Title: ", row["Title"])
  print("Publication Author:", row["Author(s)"])
  print("Publication Date:", row["Publication Date"])
  print("Journal Title:", row["Journal Title"])
  print("Cosine Similarty:", row["cosine_similarity"])
  print()


thermoregulation
Search Query: thermoregulation

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/thermoregulation-during-upper-body-exercise
Publication Title:  Thermoregulation during upper body exercise
Publication Author: {'Price, M. J.': 'https://pureportal.coventry.ac.uk/en/persons/mike-price'}
Publication Date: 1 Jan 2019
Journal Title: Journal Title Not Present
Cosine Similarty: 0.5947487828470245

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/quadriplegia-the-challenge-of-thermoregulation
Publication Title:  Quadriplegia: The challenge of thermoregulation
Publication Author: {'Price, M. J.': 'https://pureportal.coventry.ac.uk/en/persons/mike-price'}
Publication Date: Jul 2018
Journal Title: Journal Title Not Present
Cosine Similarty: 0.5514940333004401

Publication Link:   https://pureportal.coventry.ac.uk/en/publications/thermoregulation-following-spinal-cord-injury
Publication Title:  Thermoregulation following spinal cord injury


# Task 2- Clustering


In [ ]:
pip install feedparser

In [ ]:
import pandas as pd
import feedparser
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")

### Data Collection

In [ ]:
NewsFeed_health1 = feedparser.parse("https://www.bbc.co.uk/news/health/rss.xml")
hel1 = [feed.summary for feed in NewsFeed_health1.entries]
NewsFeed_health2 = feedparser.parse("http://feeds.foxnews.com/foxnews/health")
hel2 = [feed.summary for feed in NewsFeed_health2.entries]
NewsFeed_health3 = feedparser.parse("https://rss.nytimes.com/services/xml/rss/nyt/Health.xml")
hel3 = [feed.summary for feed in NewsFeed_health3.entries]
Health=hel1+hel2+hel3               #Health News
NewsFeed_ent1 = feedparser.parse("http://rss.cnn.com/rss/edition_entertainment.rss")
ent1 = [feed.summary for feed in NewsFeed_ent1.entries]
NewsFeed_ent3 = feedparser.parse("https://www.bbc.co.uk/news/entertainment_and_arts/rss.xml")
ent3 = [feed.summary for feed in NewsFeed_ent3.entries]
NewsFeed_ent2 = feedparser.parse("http://feeds.foxnews.com/foxnews/entertainment")
ent2 = [feed.summary for feed in NewsFeed_ent2.entries]
Entertainment=ent1+ent2+ent3      #Entertainment news
NewsFeed_scitech1 = feedparser.parse("http://rss.cnn.com/rss/edition_space.rss")
scitech1 = [feed.summary for feed in NewsFeed_scitech1.entries]
NewsFeed_scitech2 = feedparser.parse("http://feeds.foxnews.com/foxnews/scitech")
scitech2= [feed.summary for feed in NewsFeed_scitech2.entries]
NewsFeed_scitech3 = feedparser.parse("https://www.bbc.co.uk/news/science_and_environment/rss.xml")
scitech3= [feed.summary for feed in NewsFeed_scitech3.entries]
NewsFeed_scitech4 = feedparser.parse("https://rss.nytimes.com/services/xml/rss/nyt/Space.xml")
scitech4= [feed.summary for feed in NewsFeed_scitech4.entries]
SciTech= scitech1+ scitech2+ scitech3+scitech4  #Science and Technology news


In [ ]:
df1 = pd.DataFrame()
df1["Feed"] = Health
df1["Class"] = "Health"

df2 = pd.DataFrame()
df2["Feed"] = Entertainment
df2["Class"] = "Entertainment" 

df3 = pd.DataFrame()
df3["Feed"] =SciTech
df3["Class"] = "Sci & Tech" 

df = pd.concat([df1,df2,df3], axis=0)
df.sample(5)
df.shape

visualisation

In [ ]:
plt.figure(figsize=(20,5))
sns.countplot(y = 'Class' , data = df, palette="tab10")
plt.title("Feed's Class")
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS


comment_words = ''
stopwords = set(STOPWORDS)
  
# iterate through the csv file
for unique_key in df["Class"].unique().tolist():
  print()
  print("Category: {}\n".format(unique_key))
  for val in df[df["Class"]==unique_key]["Feed"]:
    # typecaste each val to string
    val = str(val)

    # split the value
    tokens = val.split()
      
    # Converts each token into lowercase
    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()
      
    comment_words += " ".join(tokens)+" "
    
  wordcloud = WordCloud(width = 800, height = 800,
                  background_color ='white',
                  stopwords = stopwords,
                  min_font_size = 10).generate(comment_words)
    

  # plot the WordCloud image                       
  plt.figure(figsize = (8, 8))
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
    
  plt.show()

One Hot Encoding of the classes

In [ ]:
from sklearn import preprocessing
#label Encoder

category_col = ["Class"]

df[category_col] = df[category_col].astype('|S80')

labelEncoder = preprocessing.LabelEncoder()

# creating a map of all the numerical values of each categorical labels.
mapping_dict={}
for col in category_col:
    df[col] = labelEncoder.fit_transform(df[col])
    le_name_mapping = dict(zip(labelEncoder.classes_, labelEncoder.transform(labelEncoder.classes_)))
    mapping_dict[col]=le_name_mapping
print(mapping_dict)

In [ ]:
df

Pre-processing of the feed

In [ ]:
df["Feed"] = df["Feed"].apply(lambda x: preprocess(x))

ps = PorterStemmer()

def stemming(x):
  out = []
  words = word_tokenize(x)
  for w in words:
    out.append(ps.stem(w))
  return(' '.join(out))

df["Feed"] = df["Feed"].apply(lambda x: stemming(x))

df.head()

In [ ]:
# Input and labels
X = df["Feed"]
y = df["Class"]

In [ ]:
# X into vector
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer().fit(X)
X = vectorizer.transform(X)

In [ ]:
from sklearn.cluster import KMeans

kmean = (KMeans(n_clusters = 3 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
kmean.fit(X.toarray())
kmean_labels = kmean.labels_
df["KMeans Labels"] = kmean_labels
print("K-Means labels:\n", kmean_labels)

In [ ]:
from sklearn.metrics import silhouette_score

s1 = silhouette_score(X.toarray(), kmean.fit_predict(X.toarray()))
print("Silhouette Score of K Means Clusters:",s1)

In [ ]:

from sklearn.metrics import classification_report, confusion_matrix
map = [i.decode('utf-8') for i in mapping_dict["Class"].keys()]

print("Classification Report for KMeans: \n",classification_report(df["Class"], df["KMeans Labels"], target_names= map)) 
print()

In [ ]:
cm = confusion_matrix(df["Class"], df["KMeans Labels"])
print("Confusion Matrix: " )

confusion = pd.DataFrame(cm, columns = map, index = map)

confusion

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
Z = linkage(X.toarray(), method='ward')

plt.figure(figsize=(20, 7))  
plt.title("Dendrograms: method ='ward'")  
dend = dendrogram(Z)

In [ ]:
from sklearn.cluster import AgglomerativeClustering

hierarchical_cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
hierarchical_cluster.fit_predict(X.toarray())

agglomerative_labels= hierarchical_cluster.labels_

df["Agglomerative Labels"] = agglomerative_labels
print("Agglomerative Lables:\n", agglomerative_labels)

In [ ]:
s1 = silhouette_score(X.toarray(), hierarchical_cluster.fit_predict(X.toarray()))
print("Silhouette Score of Agglomerative  Clusters:",s1)

In [ ]:
print("Classification Report for Agglomerative : \n",classification_report(df["Class"], df["Agglomerative Labels"], target_names= map)) 
print()

In [ ]:
cm = confusion_matrix(df["Class"], df["Agglomerative Labels"])
print("Confusion Matrix: " )

confusion = pd.DataFrame(cm, columns = map, index = map)

confusion

In [ ]:
# reverve decoding 
decod= mapping_dict["Class"]

docoding = decod

sample = {y : x.decode('utf-8') for x,y in docoding.items()}


for col in ["Class", "KMeans Labels", "Agglomerative Labels"]:
  df = df.replace({col: sample})

df.sample(10)

In [ ]:
testing_feed = []
test_label = []

for i, r in df.sample(20).iterrows():
  testing_feed.append(r["Feed"])
  test_label.append(r["Class"])


test_feed = pd.DataFrame()
test_feed["Feed"] = testing_feed
test_feed["Class"] = test_label

# cleaning test feed
# test_feed["Feed"] = test_feed["Feed"].apply(lambda x: feed_cleaning(x))


test_feed.head()

In [ ]:
test_feed_vector = vectorizer.transform(test_feed["Feed"])

kmean_test = []

for test in test_feed_vector:
  kmean_test.append(kmean.predict(test)[0])

test_feed["KMean Prediction"] = kmean_test

In [ ]:
# reverve decoding 
decod= mapping_dict["Class"]

docoding = decod

sample = {y : x.decode('utf-8') for x,y in docoding.items()}

for col in ["Class", "KMean Prediction"]:
  test_feed = test_feed.replace({col: sample})

test_feed

# THE END